In [ ]:
# === FULL WORKING TRAINING PIPELINE (MURA or any X-ray dataset) ===
import os, time, gc, re, torch, warnings
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
import timm
import numpy as np
warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------
PROJECT_ROOT = str(Path.cwd())
DATASET_DIRNAME = "MURA-v1.1"  # change if needed
TRAIN_IMG_PATHS_CSV = "MURA-v1.1/train_image_paths.csv"
VALID_IMG_PATHS_CSV = "MURA-v1.1/valid_image_paths.csv"
IMG_SIZE = 320
BATCH_SIZE = 34
NUM_WORKERS = min(4, os.cpu_count() or 2)
EPOCHS = 23
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", DEVICE, "| PyTorch", torch.__version__)

# ---------------- DATA PREP ----------------
def extract_bone_label(p):
    p = str(p).replace("\\", "/")
    bone = re.search(r"(XR_[A-Z]+)", p)
    study = re.search(r"(study[^/]+)", p)
    bone = bone.group(1).upper() if bone else "UNKNOWN"
    label = 1 if "positive" in (study.group(1).lower() if study else "") else 0
    return bone, label

def make_full_path(p):
    p = str(p).replace("\\", "/").lstrip("./")
    if p.startswith(DATASET_DIRNAME + "/"):
        return os.path.join(PROJECT_ROOT, p)
    elif p.startswith("train/") or p.startswith("valid/"):
        return os.path.join(PROJECT_ROOT, DATASET_DIRNAME, p)
    else:
        return os.path.join(PROJECT_ROOT, DATASET_DIRNAME, p)

def build_df(csv_path, split_name):
    df = pd.read_csv(csv_path, header=None, names=["path"], dtype=str)
    df[["bone", "label"]] = df["path"].apply(lambda x: pd.Series(extract_bone_label(x)))
    df["full_path"] = df["path"].apply(make_full_path)
    df = df[df["label"].notna()].copy().reset_index(drop=True)
    # filter missing files
    df["exists"] = df["full_path"].apply(os.path.exists)
    missing = len(df) - df["exists"].sum()
    if missing > 0:
        print(f"⚠️ {missing} missing images in {split_name} removed.")
        df = df[df["exists"]].drop(columns=["exists"])
    print(f"{split_name}: {len(df)} samples ready ✅")
    return df

train_df = build_df(TRAIN_IMG_PATHS_CSV, "train")
valid_df = build_df(VALID_IMG_PATHS_CSV, "valid")

# ---------------- DATASETS ----------------
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
valid_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

class SafeDataset(Dataset):
    def __init__(self, df, tfms):
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        path = row["full_path"]
        try:
            img = Image.open(path).convert("RGB")
            img = self.tfms(img)
        except Exception as e:
            print("⚠️ Failed to open:", path, "err:", e)
            img = torch.zeros(3, IMG_SIZE, IMG_SIZE)
        return img, torch.tensor(int(row["label"]))

def make_sampler(df):
    counts = df["label"].value_counts().to_dict()
    weights = df["label"].map(lambda x: 1.0/counts[x])
    return WeightedRandomSampler(torch.DoubleTensor(weights.values),
                                 num_samples=len(weights), replacement=True)

train_loader = DataLoader(SafeDataset(train_df, train_tfms),
                          batch_size=BATCH_SIZE, sampler=make_sampler(train_df),
                          num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(SafeDataset(valid_df, valid_tfms),
                          batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

print("📦 Train batches:", len(train_loader), "| Valid batches:", len(valid_loader))

# ---------------- MODEL ----------------
## model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=2)


################ chnages ######################################################
# Example safe access
if hasattr(model, "classifier"):
    in_feats = model.classifier.in_features
    model.classifier = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, 2))
elif hasattr(model, "fc"):
    in_feats = model.fc.in_features
    model.fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, 2))
elif hasattr(model, "head"):
    # some timm models use .head
    try:
        in_feats = model.head.in_features
        model.head = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, 2))
    except Exception:
        pass

#################################################################################

#####  model.fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(num_ftrs, 2))
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler("cuda")


############################################################################################N
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)





##################################################################################


# ---------------- TRAINING ----------------
best_acc = 0
for epoch in range(EPOCHS):
    model.train()
    total_loss, total_correct, total = 0, 0, 0
    pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{EPOCHS}")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
         ##############################################################################################
        mixed_imgs, y_a, y_b, lam = mixup_data(imgs, labels)
       
        with torch.amp.autocast(device_type="cuda"):
             out = model(mixed_imgs)2
             loss = mixup_criterion(criterion, out, y_a, y_b, lam)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * imgs.size(0)
        total_correct += (out.argmax(1) == labels).sum().item()
        total += imgs.size(0)
        pbar.set_postfix(loss=f"{total_loss/total:.4f}", acc=f"{100*total_correct/total:.2f}%")
    scheduler.step()

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad(), torch.amp.autocast(device_type="cuda"):
        for imgs, labels in valid_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            out = model(imgs)
            val_correct += (out.argmax(1) == labels).sum().item()
            val_total += labels.size(0)
    val_acc = 100 * val_correct / val_total
    print(f"📈 Epoch {epoch+1} | Val Accuracy: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "efficientnet_b3.pth")
        print(f"💾 Saved best model (acc={best_acc:.2f}%)")

print(f"\n✅ Training complete. Best validation accuracy: {best_acc:.2f}%")


/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/torch-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Device: cuda | PyTorch 2.9.1+cu126
train: 11953 samples ready ✅
valid: 3197 samples ready ✅
📦 Train batches: 352 | Valid batches: 95


Train Epoch 1/23:  40%|████      | 142/352 [11:00<14:23,  4.11s/it, acc=59.90%, loss=0.6034]

⚠️ Failed to open: /mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image3.png err: cannot identify image file '/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image3.png'


Train Epoch 1/23:  81%|████████▏ | 286/352 [21:05<04:39,  4.23s/it, acc=61.37%, loss=0.5745]

...

Train Epoch 7/23:  22%|██▏       | 79/352 [05:28<18:44,  4.12s/it, acc=69.40%, loss=0.3887]

⚠️ Failed to open: /mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png err: cannot identify image file '/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png'


Train Epoch 7/23: 100%|██████████| 352/352 [24:09<00:00,  4.12s/it, acc=70.80%, loss=0.3909]


📈 Epoch 7 | Val Accuracy: 76.38%


Train Epoch 8/23:   3%|▎         | 9/352 [00:43<24:51,  4.35s/it, acc=71.57%, loss=0.3904]

In [7]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import timm
import re

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_WEIGHTS = "efficientnet_b3.pth"
IMAGE_PATH = "/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/valid/XR_SHOULDER/patient11245/study1_positive/image1.png"
IMG_SIZE = 320     # must match your training IMG_SIZE

# -------------------------------
# SAME TRANSFORMS USED IN TRAINING
# -------------------------------
tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# -------------------------------
# EXTRACT BONE TYPE FROM PATH
# -------------------------------
def extract_bone(path):
    path = path.replace("\\", "/")
    match = re.search(r"(XR_[A-Z]+)", path)
    if match:
        return match.group(1)
    return "UNKNOWN"

# -------------------------------
# LOAD MODEL (CORRECT ARCHITECTURE)
# -------------------------------
def load_model():
    model = timm.create_model(
        "efficientnet_b3",
        pretrained=False,
        num_classes=2
    )

    # Fix classifier exactly as training
    if hasattr(model, "classifier"):
        in_feats = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_feats, 2)
        )
    elif hasattr(model, "fc"):
        in_feats = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_feats, 2)
        )
    elif hasattr(model, "head"):
        in_feats = model.head.in_features
        model.head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_feats, 2)
        )

    model.load_state_dict(torch.load(MODEL_WEIGHTS, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    print("✅ EfficientNet-B3 Loaded Successfully")
    return model

# -------------------------------
# PREDICT
# -------------------------------
def predict(image_path, model):
    img = Image.open(image_path).convert("RGB")
    img = tfms(img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        out = model(img)
        prob = torch.softmax(out, dim=1)
        pred = prob.argmax(dim=1).item()

    return pred, prob.cpu().numpy()

# -------------------------------
# RUN
# -------------------------------
if __name__ == "__main__":
    bone_type = extract_bone(IMAGE_PATH)

    model = load_model()
    pred, prob = predict(IMAGE_PATH, model)

    anomaly_map = {0: "NEGATIVE (Normal)", 1: "POSITIVE (Abnormal)"}

    print("\n===============================")
    print("Bone Type:", bone_type)
    print("Anomaly:", anomaly_map[pred])
    print("Probabilities:", prob)
    print("===============================\n")


✅ EfficientNet-B3 Loaded Successfully

Bone Type: XR_SHOULDER
Anomaly: NEGATIVE (Normal)
Probabilities: [[0.6723247 0.3276753]]

